<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/10_19_22_SBERT%2B_BigGraph_Embeddings_NN_gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 1.9 MB 57.1 MB/s 
     |████████████████████████████████| 5.3 MB 50.6 MB/s 
     |████████████████████████████████| 1.3 MB 55.9 MB/s 
     |████████████████████████████████| 163 kB 72.0 MB/s 
     |████████████████████████████████| 7.6 MB 44.9 MB/s 
     |████████████████████████████████| 162 kB 35.8 MB/s 
     |████████████████████████████████| 182 kB 38.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 52.6 MB/s 
     |████████████████████████████████| 158 kB 47.8 MB/s 
     |████████████████████████████████| 157 kB 34.5 MB/s 
     |████████████████████████████████| 157 kB 59.5 MB/s 
     |████████████████████████████████| 157 kB 48.7 MB/s 
     |████████████████████████████████| 157 kB 12.5 MB/s 
     |█████████████████████████████

The notebook for Gossipcop. (Similar one present for Politifact is already saved in the Drive and on Github) 

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

In [3]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Mounted at /content/drive


In [5]:
dataset_id = 'gossipcop'

In [6]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv", converters=converters)
df.head()

,title,text,tweets,retweets,label,url,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Kendall Kylie Jenner Jenner NOT Upset Up...,NaN,[],"[995423424741888001, 995461685166202880, 99987...",fake,NaN,3,1.386294,0,0.000000
1,Kim Kardashian Dethroned Dethroned By Khlo...,NaN,[],"[848843565027516416, 849030801970868224, 84884...",fake,NaN,3,1.386294,0,0.000000
2,Kim Kardashian Did NOT Hot Staffer Hot Sta...,NaN,[],"[940685393112064001, 977921622672920576, 94031...",fake,NaN,8,2.197225,0,0.000000
3,The Voice The Voice Team NOT Surprised Sur...,NaN,[],[],fake,NaN,0,0.000000,0,0.000000
4,Drake NOT Angelina Jolie s Toy Boy Toy Boy ...,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","[948022124626808832, 948630026496323585, 94801...",fake,NaN,18,2.944439,7,2.079442


# Using the embeddings for title and text from previous experiments

In [7]:
title_embedding_file = base_dir/f"{dataset_id}_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    titles = df.title.tolist()
    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [8]:
text_embedding_file = base_dir/f"{dataset_id}_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    texts = (df.title + "\n" + df.text).tolist()
    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

# Training - Saving the machine learning model specifications

In [9]:
import tensorflow as tf

In [10]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

In [11]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

### PyTorch_BigGraph embeddings

In [12]:
edim = 128
bg_embeddings = np.load(base_dir/f'{dataset_id}_pt_biggraph_article_embeddings_{edim}.npy')
idx = np.load(base_dir/f"{dataset_id}_pt_biggraph_article_idx_{edim}.npy")

In [13]:
graph_embeddings = np.zeros((text_embeddings.shape[0], edim))
graph_embeddings[idx] = bg_embeddings

In [14]:
params = {
    "epochs":200,
    "bs":128,
    # 'lr':1e-3
}

In [15]:
X = np.concatenate([
    text_embeddings, 
    df.log_num_retweets.to_numpy()[..., None], 
    df.log_num_tweets.to_numpy()[..., None],
    graph_embeddings], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 898), (19968,))

In [16]:
skf = StratifiedKFold(shuffle=True, random_state=124)

Creating a group and logging all train, test data in wandb for better tracking and visualisation.

In [17]:
WANDB_ENTITY = 'saloniteam'
WANDB_PROJECTS = 'nofolds'

GROUP = f"{dataset_id}-fulltext-sbert+tw+ptbg128-mlp"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_ENTITY, group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/200
118/125 [===========================>..] - ETA: 0s - loss: 0.4825 - accuracy: 0.7508

...

wandb: Adding directory to artifact (/content/wandb/run-20221019_105142-3t1wmuk7/files/model-best)... Done. 0.1s


125/125 [==============================] - 1s 11ms/step - loss: 0.2268 - accuracy: 0.9073 - val_loss: 0.2899 - val_accuracy: 0.8748
Epoch 43/200
120/125 [===========================>..] - ETA: 0s - loss: 0.2222 - accuracy: 0.9092

wandb: Adding directory to artifact (/content/wandb/run-20221019_105142-3t1wmuk7/files/model-best)... Done. 0.1s


125/125 [==============================] - 1s 11ms/step - loss: 0.2221 - accuracy: 0.9090 - val_loss: 0.2871 - val_accuracy: 0.8786
Epoch 44/200
125/125 [==============================] - 1s 5ms/step - loss: 0.2204 - accuracy: 0.9106 - val_loss: 0.2931 - val_accuracy: 0.8728
Epoch 45/200
125/125 [==============================] - 1s 4ms/step - loss: 0.2181 - accuracy: 0.9101 - val_loss: 0.2904 - val_accuracy: 0.8696
Epoch 46/200
125/125 [==============================] - 1s 4ms/step - loss: 0.2198 - accuracy: 0.9086 - val_loss: 0.2936 - val_accuracy: 0.8793
Epoch 47/200
125/125 [==============================] - 1s 5ms/step - loss: 0.2162 - accuracy: 0.9108 - val_loss: 0.2922 - val_accuracy: 0.8726
Epoch 48/200
125/125 [==============================] - 1s 5ms/step - loss: 0.2146 - accuracy: 0.9122 - val_loss: 0.2968 - val_accuracy: 0.8653
Epoch 49/200
125/125 [==============================] - 1s 4ms/step - loss: 0.2128 - accuracy: 0.9134 - val_loss: 0.2928 - val_accuracy: 0.8753
Epoc

accuracy,▁▂▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▇▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▆▇▇▇█▇█▆▇▇█▇█▆▇▇▇▆▇▇▇▇▇▆▆▇▇█▇▇▇▇▆▇▆▇▇
val_loss,█▅▃▃▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▃▃▃▃▄▄▄▄▅▄▄▅▅▅▅▅
accuracy,0.95549
best_epoch,42
